# Cinetel Data Cleaning

Cinetel data scraped daily from their boxoffice published everyday at this [link](https://www.cinetel.it/pages/boxoffice.php?edperiodo=aWVyaQ==).

## Reading In Files

In [34]:
import pandas as pd

In [35]:
df1 = pd.read_csv(r"E:\data_analysis_python\cinetel\cinetel_.csv")
df1.head()

,Pos.,Titolo,Prima Progr.,Nazione,Distribuzione,Incasso,Presenze,Incasso al 16/11/2023,Presenze al 16/11/2023,2023-11-17 15:31:50.351708
0,1,C'E' ANCORA DOMANI,'2023-10-26'26/10/2023,ITA,VISION DISTRIBUTION,449991.06,66700,15231636.85,2202169,2023-11-17 15:31:50.351708
1,2,HUNGER GAMES - LA BALLATA DELL'USIGNOLO E DEL ...,'2023-11-15'15/11/2023,USA,MEDUSA FILM S.P.A.,238466.31,32469,584902.45,79311,2023-11-17 15:31:50.351708
2,3,THE MARVELS,'2023-11-08'08/11/2023,USA,WALT DISNEY S.M.P. ITALIA,60914.35,8628,2260142.44,305291,2023-11-17 15:31:50.351708
3,4,THANKSGIVING,'2023-11-16'16/11/2023,USA,EAGLE PICTURES S.P.A.,41232.57,5709,41232.57,5709,2023-11-17 15:31:50.351708
4,5,THE OLD OAK,'2023-11-16'16/11/2023,FRA,LUCKY RED DISTRIB.,36605.89,5879,56540.79,8609,2023-11-17 15:31:50.351708


---

In [36]:
df1 = pd.DataFrame(df1)

## Cleaning the dataframe

To clean the dataframe I decided to drop duplicate values and columns that aren't relevant to my analysis, such as `df1['Incasso al 16/11/2023']` and `df1['Presenze al 16/11/2023]`. I then decided to rename the existing columns using the snake_case rule in order to make the analysis easier.

In [37]:
df1 = df1.drop_duplicates()

In [38]:
# df1 = df1.drop(columns= ['Incasso al 16/11/2023', 'Presenze al 16/11/2023'])

# at first i thought these columns weren't relevant
# making visualizations, i realized i need the data to have accurate insights on movies that started their screenings before i started to scrape the data

In [39]:
df1 = df1.rename(columns = {'Pos.':'daily_rank', 'Titolo' : 'title', 'Prima Progr.' : 'first_screening_date', 'Nazione' : 'nation', 'Distribuzione' : 'distribution', 'Incasso' : 'daily_takings', 'Presenze' : 'daily_attendance','Incasso al 16/11/2023' : 'total_takings',  'Presenze al 16/11/2023' : 'total_attendance', '2023-11-17 15:31:50.351708' : 'date_pulled'})

In [40]:
# dropping the extra 'header row' that is imported everyday when the .csv file gets updated
df1 = df1[df1['daily_rank'].str.contains('Pos.')==False]

In [41]:
# making the 'title' and 'distribution' entries consistent by appling the title format
df1['title'] = df1['title'].apply(lambda x: x.title())
df1['distribution'] = df1['distribution'].apply(lambda x: x.title())

In [42]:
# cleaning the 'first_screening_date' entries
df1['first_screening_date'] = df1['first_screening_date'].str[12:]

In [43]:
# cleaning the column showing the date the data was pulled from the site
df1['date_pulled'] = pd.to_datetime(df1['date_pulled'])
df1['date_pulled'] = pd.to_datetime(df1['date_pulled']).dt.date

#extra tries:
#df1['date_pulled'] = df1['date_pulled'].str[:10]
#df1['date_pulled'] = df1['date_pulled'].str.replace('[^a-zA-Z0-9]', '', regex = True)
#df1['date_pulled'] = df1['date_pulled'].apply(lambda x: str(x))
#df1['date_pulled'] = df1['date_pulled'].apply(lambda x: x[0:2] + '/' + x[2:4] + '/' + x[4:8])

In [44]:
from datetime import timedelta, datetime

In [45]:
df1['first_screening_date']

0       26/10/2023
1       15/11/2023
2       08/11/2023
3       16/11/2023
           ...    
1348    14/03/2024
1349    27/02/2024
1350    07/03/2024
1351    25/01/2024
Name: first_screening_date, Length: 1230, dtype: object

In [46]:
# formatting
df1['first_screening_date'] = pd.to_datetime(df1['first_screening_date'], errors='coerce')
df1['first_screening_date'] = pd.to_datetime(df1['first_screening_date']).dt.date

C:\Users\Franci\AppData\Local\Temp\ipykernel_14644\2741308956.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df1['first_screening_date'] = pd.to_datetime(df1['first_screening_date'], errors='coerce')


In [47]:
df1['daily_takings'] = df1['daily_takings'].astype(float)
df1['daily_attendance'] = df1['daily_attendance'].astype(int)
df1['daily_rank'] = df1['daily_rank'].astype(int)
df1['total_takings'] = df1['total_takings'].astype(float)
df1['total_attendance'] = df1['total_attendance'].astype(int)

---

## Data manipulation

I decided to create new columns to get information I may need later on during the analysis.

In [48]:
# calculating the number of days of screenings for each movie 
df1['screening_days'] = datetime.today().date() - df1['first_screening_date']
df1['screening_days'] = df1['screening_days'].map(lambda x: str(x)[:-14])

In [49]:
df1['screening_days']

0       148
1       128
2       135
3       127
       ... 
1348      8
1349     24
1350     15
1351     57
Name: screening_days, Length: 1230, dtype: object

In [50]:
#df1['screening_days'] = df1['screening_days'].str.strip()
df1['screening_days'] = df1['screening_days'].replace('', '0')
df1['screening_days'] = df1['screening_days'].astype(int) + 1

In [51]:
# creating a column to show the date the 'daily_takings' and 'daily_attendace' refer to
df1['date'] = df1['date_pulled'] - timedelta(days=1)
df1['date'] = pd.to_datetime(df1['date'])

In [52]:
# creating a column showing the day of the week. it'll be useful for further analysis
df1['day_of_week'] = df1['date'].dt.day_name()

In [53]:
df1['avg_ticket_price'] = df1['daily_takings'] / df1['daily_attendance']

In [54]:
pd.set_option('display.max_rows', 8)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
df1

,daily_rank,title,first_screening_date,nation,distribution,daily_takings,daily_attendance,total_takings,total_attendance,date_pulled,screening_days,date,day_of_week,avg_ticket_price
0,1,C'E' Ancora Domani,2023-10-26,ITA,Vision Distribution,449991.06,66700,15231636.85,2202169,2023-11-17,149,2023-11-16,Thursday,6.75
1,2,Hunger Games - La Ballata Dell'Usignolo E Del ...,2023-11-15,USA,Medusa Film S.P.A.,238466.31,32469,584902.45,79311,2023-11-17,129,2023-11-16,Thursday,7.34
2,3,The Marvels,2023-11-08,USA,Walt Disney S.M.P. Italia,60914.35,8628,2260142.44,305291,2023-11-17,136,2023-11-16,Thursday,7.06
3,4,Thanksgiving,2023-11-16,USA,Eagle Pictures S.P.A.,41232.57,5709,41232.57,5709,2023-11-17,128,2023-11-16,Thursday,7.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1348,7,Imaginary,2024-03-14,USA,Eagle Pictures S.P.A.,13563.99,1859,384611.03,50437,2024-03-22,9,2024-03-21,Thursday,7.30
1349,8,Food For Profit,2024-02-27,ITA,Mescalito Film,11987.59,1670,138756.45,19103,2024-03-22,25,2024-03-21,Thursday,7.18
1350,9,Un Altro Ferragosto,2024-03-07,ITA,01 Distribution,11285.66,1800,1466064.62,217411,2024-03-22,16,2024-03-21,Thursday,6.27
1351,10,Povere Creature! (Poor Things),2024-01-25,GBR,Walt Disney S.M.P. Italia,9356.84,1468,8994034.30,1255980,2024-03-22,58,2024-03-21,Thursday,6.37


---

## Export as .csv

In [55]:
import os

In [56]:
clean_file_name = 'cinetel_clean.csv'
clean_file_path = os.path.join(os.getcwd(), clean_file_name)

In [57]:
df1.to_csv(clean_file_path, index=False)
print(f"Il DataFrame è stato esportato con successo in {clean_file_path}.")

Il DataFrame è stato esportato con successo in e:\data_analysis_python\cinetel\cinetel_clean.csv.


---

## Data Exploration

In [58]:
import seaborn as sns
import matplotlib.pyplot as plt

In [59]:
pd.set_option('display.max_rows', 8)

In [60]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1230 entries, 0 to 1351
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   daily_rank            1230 non-null   int32         
 1   title                 1230 non-null   object        
 2   first_screening_date  1223 non-null   object        
 3   nation                1230 non-null   object        
 4   distribution          1230 non-null   object        
 5   daily_takings         1230 non-null   float64       
 6   daily_attendance      1230 non-null   int32         
 7   total_takings         1230 non-null   float64       
 8   total_attendance      1230 non-null   int32         
 9   date_pulled           1230 non-null   object        
 10  screening_days        1230 non-null   int32         
 11  date                  1230 non-null   datetime64[ns]
 12  day_of_week           1230 non-null   object        
 13  avg_ticket_price      1

In [61]:
df1.describe()

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,screening_days,date,avg_ticket_price
count,1230.00,1230.00,1230.00,1230.00,1230.00,1230.00,1230,1230.00
mean,5.50,139734.72,19431.80,4668944.24,661908.44,87.01,2024-01-16 01:57:04.390243840,6.91
min,1.00,6272.70,864.00,11161.11,1470.00,1.00,2023-11-16 00:00:00,3.70
25%,3.00,29065.72,4410.50,479539.48,66511.00,58.00,2023-12-16 00:00:00,6.35
50%,5.50,66978.01,9855.50,1684052.96,243103.00,83.00,2024-01-14 00:00:00,6.92
75%,8.00,162093.96,22927.00,4557664.10,647994.00,114.00,2024-02-15 00:00:00,7.39
max,10.00,1544231.00,211764.00,36581221.80,5399550.00,919.00,2024-03-21 00:00:00,21.18
std,2.87,189728.40,25402.17,8142493.27,1183090.05,63.47,NaN,1.07


In [62]:
df1.isnull().sum()

daily_rank              0
title                   0
first_screening_date    7
nation                  0
                       ..
screening_days          0
date                    0
day_of_week             0
avg_ticket_price        0
Length: 14, dtype: int64

In [63]:
df1.nunique()

daily_rank                10
title                    102
first_screening_date      54
nation                    12
                        ... 
screening_days            54
date                     117
day_of_week                7
avg_ticket_price        1179
Length: 14, dtype: int64

In [64]:
df1.corr(numeric_only=True)

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,screening_days,avg_ticket_price
daily_rank,1.00,-0.54,-0.55,-0.06,-0.06,0.12,-0.12
daily_takings,-0.54,1.00,1.00,0.12,0.12,0.05,0.24
daily_attendance,-0.55,1.00,1.00,0.14,0.14,0.05,0.20
total_takings,-0.06,0.12,0.14,1.00,1.00,0.34,-0.18
total_attendance,-0.06,0.12,0.14,1.00,1.00,0.34,-0.19
screening_days,0.12,0.05,0.05,0.34,0.34,1.00,-0.04
avg_ticket_price,-0.12,0.24,0.20,-0.18,-0.19,-0.04,1.00


In [65]:
df1.groupby('title').mean(numeric_only=True).sort_values(by = 'total_attendance', ascending= False)

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,screening_days,avg_ticket_price
title,,,,,,,
C'E' Ancora Domani,4.66,247731.30,36798.43,30931682.52,4533157.52,149.00,6.24
Oppenheimer,8.38,25121.14,3891.00,28329103.11,3799378.88,213.00,6.29
Wonka,3.85,368953.34,48842.15,9842301.36,1311072.30,100.00,7.32
Dune,9.00,25782.87,3338.75,7432259.33,1035522.25,919.00,7.53
...,...,...,...,...,...,...,...
Royal Opera House 2023/2024,7.00,19784.50,1916.00,35873.50,3496.00,185.00,10.33
Seventeen Tour Follow To Japan: Live Viewing,9.00,66089.87,3121.00,66089.87,3121.00,98.00,21.18
Casanova Operapop - Il Film,9.00,12093.47,1256.33,25578.93,2715.33,117.00,9.62
Another End,6.00,16447.35,2689.00,16447.35,2697.00,1.00,6.12


In [66]:
df2 = df1.groupby('title').mean(numeric_only=True).sort_values(by = 'total_attendance', ascending= False)
df2

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,screening_days,avg_ticket_price
title,,,,,,,
C'E' Ancora Domani,4.66,247731.30,36798.43,30931682.52,4533157.52,149.00,6.24
Oppenheimer,8.38,25121.14,3891.00,28329103.11,3799378.88,213.00,6.29
Wonka,3.85,368953.34,48842.15,9842301.36,1311072.30,100.00,7.32
Dune,9.00,25782.87,3338.75,7432259.33,1035522.25,919.00,7.53
...,...,...,...,...,...,...,...
Royal Opera House 2023/2024,7.00,19784.50,1916.00,35873.50,3496.00,185.00,10.33
Seventeen Tour Follow To Japan: Live Viewing,9.00,66089.87,3121.00,66089.87,3121.00,98.00,21.18
Casanova Operapop - Il Film,9.00,12093.47,1256.33,25578.93,2715.33,117.00,9.62
Another End,6.00,16447.35,2689.00,16447.35,2697.00,1.00,6.12
